https://github.com/JeongJaeyoung0/crawling/blob/bb0d95279eb33b1e0b2be1e9e474adacafa91b68/210507_crawler_naver%20blog.ipynb

In [1]:
import sys    # 시스템
import os     # 시스템

import pandas as pd    # 판다스 : 데이터분석 라이브러리
import numpy as np     # 넘파이 : 숫자, 행렬 데이터 라이브러리

from bs4 import BeautifulSoup     # html 데이터 전처리
from selenium import webdriver    # 웹 브라우저 자동화

import time                       # 시간 지연
from tqdm import tqdm_notebook    # 진행상황 표시

In [3]:
# 검색어
keyword1 = input("1.크롤링할 키워드를 입력하세요: ")

'''
#검색할 키워드 정리
10대 여자 어린이날 선물
10대 남자 어린이날 선물

2~30대 여자 어버이날 선물
2~30대 남자 어버이날 선물
4~50대 여자 어버이날 선물
4~50대 남자 어버이날 선물
60대 여자 어버이날 선물
60대 남자 어버이날 선물

2~30대 여자 스승의날 선물
2~30대 남자 스승의날 선물
4~50대 여자 스승의날 선물
4~50대 남자 스승의날 선물
60대 여자 스승의날 선물
60대 남자 스승의날 선물

10대 여자 생일 선물
10대 남자 생일 선물
2~30대 여자 생일 선물
2~30대 남자 생일 선물
4~50대 여자 생일 선물
4~50대 남자 생일 선물
60대 여자 생일 선물
60대 남자 생일 선물

2~30대 여자 결혼기념일 선물
2~30대 남자 결혼기념일 선물
4~50대 여자 결혼기념일 선물
4~50대 남자 결혼기념일 선물
60대 여자 결혼기념일 선물
60대 남자 결혼기념일 선물
'''

1.크롤링할 키워드를 입력하세요: 10대 어린이날 선물


'\n10대 여자 어린이날 선물\n10대 남자 어린이날 선물\n\n2~30대 여자 어버이날 선물\n2~30대 남자 어버이날 선물\n4~50대 여자 어버이날 선물\n4~50대 남자 어버이날 선물\n60대 여자 어버이날 선물\n60대 남자 어버이날 선물\n\n2~30대 여자 스승의날 선물\n2~30대 남자 스승의날 선물\n4~50대 여자 스승의날 선물\n4~50대 남자 스승의날 선물\n60대 여자 스승의날 선물\n60대 남자 스승의날 선물\n\n10대 여자 생일 선물\n10대 남자 생일 선물\n2~30대 여자 생일 선물\n2~30대 남자 생일 선물\n4~50대 여자 생일 선물\n4~50대 남자 생일 선물\n60대 여자 생일 선물\n60대 남자 생일 선물\n\n2~30대 여자 결혼기념일 선물\n2~30대 남자 결혼기념일 선물\n4~50대 여자 결혼기념일 선물\n4~50대 남자 결혼기념일 선물\n60대 여자 결혼기념일 선물\n60대 남자 결혼기념일 선물\n'

수집할 url 크롤링

In [4]:
# 크롬 웹브라우저 실행
driver = webdriver.Chrome(r"D:\정보현\chromedriver.exe")

# 사이트 주소
driver.get("http://www.naver.com")
time.sleep(1)

# 검색창에 '검색어' 검색
element = driver.find_element_by_id("query") #f12눌러서 확인
element.send_keys(keyword1)
element.submit()
time.sleep(1)

# 'VIEW' 클릭
driver.find_element_by_link_text("VIEW").click()

# '블로그' 클릭
driver.find_element_by_link_text("블로그").click()
time.sleep(1)

# 스크롤 다운
def scroll_down(driver):
    driver.execute_script("window.scrollTo(0, 99999999)")
    time.sleep(1)

# n: 스크롤할 횟수 설정
n = 10
i = 0
while i < n:
    scroll_down(driver)
    i = i+1
    
# 블로그 글 url들 수집
url_list = []

# URL_raw 크롤링 시작
article_raw = driver.find_elements_by_css_selector(".api_txt_lines.total_tit")
article_raw[0]

# 크롤링한 url 정제 시작
for article in article_raw:
    url = article.get_attribute('href')   
    url_list.append(url)

time.sleep(1)
    
print("url개수: ", len(url_list))

df = pd.DataFrame({'url':url_list})
df

# url_list 저장
df.to_excel("naver_blog_url.xlsx")

url개수:  210


블로그 내용 크롤링

In [5]:
import sys
import os

import pandas as pd
import numpy as np

In [6]:
# "url_list.csv" 불러오기
url_load = pd.read_excel("naver_blog_url.xlsx")

num_list = len(url_load)

print(num_list)
#url_load

210


In [7]:
dict = {}    # 전체 크롤링 데이터를 담을 그릇

number = num_list    # 수집할 글 갯수

# 수집한 url 돌면서 데이터 수집
for i in tqdm_notebook(range(0, number)):
    # 글 띄우기
    url = url_load["url"][i]
    driver = webdriver.Chrome(r"D:\정보현\chromedriver.exe")
    driver.get(url)   # 글 띄우기
    
    # 크롤링
    
    try : 
        # 글의 iframe 접근
        driver.switch_to.frame("mainFrame")

        target_info = {}  # 개별 블로그 내용을 담을 딕셔너리 생성
        
        # 내용 크롤링
        overlays = ".se-component.se-text.se-l-default"                                 
        contents = driver.find_elements_by_css_selector(overlays)

        content_list = []
        for content in contents:
            content_list.append(content.text)
 
        content_str = ' '.join(content_list)
    
        # 크롤링한 글은 target_info라는 딕셔너리에 담음
        target_info['content'] = content_str
        

        # 각각의 글은 dict라는 딕셔너리에 담음
        dict[i] = target_info
        time.sleep(1)


        # 글 하나 크롤링 후 크롬 창 닫기
        driver.close()       
    
    # 에러나면 현재 크롬창을 닫고 다음 글(i+1)로 이동
    except:
        driver.close()
        time.sleep(1)
        continue
    
    # 중간 저장 (없어도됨)
    if i == 30 or 50 or 80:
        # 판다스로 만들기
        import pandas as pd
        result_df = pd.DataFrame.from_dict(dict, 'index')

        # 저장하기
        result_df.to_excel("naver_blog_content.xlsx", encoding='utf-8-sig')   # 한글은 오류날 수 있어서 encoding 추가
        time.sleep(3)

print('수집한 글 갯수: ', len(dict))
#print(dict)

C:\Users\minno\AppData\Local\Temp\ipykernel_9836\2700816108.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(range(0, number)):


  0%|          | 0/210 [00:00<?, ?it/s]

수집한 글 갯수:  206


In [8]:
# 판다스화
result_df = pd.DataFrame.from_dict(dict, 'index')
#result_df

# 엑셀 저장
result_df.to_excel("naver_blog_content.xlsx", encoding='utf-8-sig')

액셀 데이터 텍스트 파일로 변환하기

In [9]:
#엑셀 파일을 읽어오는 소스 코드는 
tf=pd.read_excel('naver_blog_content.xlsx', usecols = ['content'])
tf.to_csv("content.txt", index=True)

형태소 분석

In [10]:
#파일 오픈
of = open("content.txt","r",encoding="utf-8")
data=of.read()
data

',content\n0,"#쥬라기월드 도미니언영화포스터\n#어린이날선물\n#10대남자선물 벌써 5월이 시작되었습니다.^^\n그리고 오늘 5월2일(월)부터 실외마스크 착용의무가 완화된다고 합니다.\n2020년 10월 이후 566일만의 일인데요.\n그 동안 얼마나 답답한 날들이 이어졌나요.\n그런데 이렇게 마스크 착용이 완화되어도\n2년동안 익숙해지기도 했고, 위험함을 알기에\n계속 마스크를 쓴다는 의견들도 많습니다.\n물론 길거리에 착용하지 않는 사람들도 늘겠지만요.\n우리 모두의 안전을 위해 힘쓰는 건 당연하기에\n개인적인 관리로 건강 유의하시길 바랍니다. \n그럼 오늘의 포스팅 시작하겠습니다.\n역대급 기대작이라고 하는 \'쥬라기월드 도미니언\'을 소개합니다. 블랙위도우 영화 간략 정보\n개봉\n2022.06.01.\n등급\n12세 관람가\n장르\n액션, 모험\n국가\n미국\n러닝타임\n147분\n줄거리-네이버영화-\n공룡들의 터전이었던 이슬라 누블라 섬이 파괴된 후, \n마침내 공룡들은 섬을 벗어나 세상 밖으로 출몰한다.\n 지상에 함께 존재해선 안 될 위협적 생명체인 공룡의 등장으로 \n인류 역사상 겪어보지 못한 사상 최악의 위기를 맞이한 인간들. \n지구의 최상위 포식자 자리를 걸고 인간과 공룡의 최후의 사투가 펼쳐진다.      \'쥬라기월드 도미니언영화포스터’ 를\n강력추천하며\n이번 포스팅은 마무리하겠습니다💕 ▼▼주문은 여기서▼▼\n▼▼포토판다 주문관련 포스팅 글▼▼ "\n1,"[돈 잘버는 회사 가치투자 9회] 어린이날 선물로 주식 사달라고 하는 다음세대 10대 아이들 \n- 강태훈 선생\n5월은 가정의 달인데요. 제가 가르치는 10대 아이들이 어린이날 선물로 주식 사달라고 하더라고요. ^^; 왠 뜬금없이 주식 선물 타령이라고 하니깐요. 용돈으로 받는 것보다도 주식으로 받는 것이 미래에 부자될것 같다라고 이야기 하더라고요.\n유태인들은 13세에 성인식을 하면 그때 주식계좌를 부모님이 자녀들에게 선물해 준다고 하더라고요. 사춘기가 되면서 사회적 책임

https://thinkwarelab.wordpress.com/2016/08/30/%ED%8C%8C%EC%9D%B4%EC%8D%AC-%ED%98%95%ED%83%9C%EC%86%8C-%EB%B6%84%EC%84%9D%EC%9C%BC%EB%A1%9C-%EC%9B%8C%EB%93%9C%ED%81%B4%EB%9D%BC%EC%9A%B0%EB%93%9C-%EA%B7%B8%EB%A6%AC%EA%B8%B0/

In [11]:
!pip install konlpy
from konlpy.tag import Okt
okt=Okt()  #Okt 객체를 생성

nouns = okt.nouns(data)

In [12]:
#명사 추출
nouns

['도미니언',
 '영화포스터',
 '벌써',
 '시작',
 '오늘',
 '월',
 '부터',
 '실외',
 '마스크',
 '착용',
 '의무',
 '완화',
 '이후',
 '그',
 '동안',
 '얼마나',
 '날',
 '마스크',
 '착용',
 '완화',
 '기도',
 '계속',
 '마스크',
 '의견',
 '길거리',
 '착용',
 '사람',
 '요',
 '우리',
 '모두',
 '안전',
 '위해',
 '건',
 '개인',
 '관리',
 '건강',
 '유의',
 '오늘',
 '포스팅',
 '시작',
 '역대',
 '대작',
 '쥬라기',
 '월드',
 '도미니언',
 '소개',
 '블랙위도우',
 '영화',
 '간략',
 '정보',
 '개봉',
 '등급',
 '세',
 '관람',
 '장르',
 '액션',
 '모험',
 '국가',
 '미국',
 '러닝',
 '타임',
 '줄거리',
 '네이버',
 '영화',
 '공룡',
 '터전',
 '이슬라',
 '누블',
 '섬',
 '파괴',
 '후',
 '마침내',
 '공룡',
 '섬',
 '세상',
 '밖',
 '출몰',
 '지상',
 '존재',
 '선',
 '안',
 '위협',
 '생명체',
 '공룡',
 '등장',
 '인류',
 '역사상',
 '보지',
 '못',
 '사상',
 '최악',
 '위기',
 '맞이',
 '인간',
 '지구',
 '최',
 '상위',
 '포식자',
 '자리',
 '인간',
 '공룡',
 '최후',
 '사투',
 '쥬라기',
 '월드',
 '도미니언',
 '영화포스터',
 '를',
 '강력',
 '추천',
 '이번',
 '포스팅',
 '마무리',
 '주문',
 '여기',
 '포토',
 '판다',
 '주문',
 '관련',
 '포스팅',
 '글',
 '돈',
 '잘버',
 '회사',
 '가치투자',
 '어린이날',
 '선물',
 '주식',
 '달라',
 '다음',
 '세대',
 '아이',
 '강태훈',
 '선생',
 '가정',
 '제',
 '아이',
 '어린이날',
 '선물',
 '주식'

In [66]:
#불용어 처리
stopword = ['선물','어린이날','날','여자','아이','어린이','추천', '엄마', '나', '구매', '때', '수', '것', '더', '이', '저', '가격', '말', 
            '그', '생각', '사용', '제', '제품', '우리', '친구', '거', '조카', '요즘', '등', '위', '요', '집', '하나', '준비', '전', '가지', 
           '곳', '중', '때문', '사진']
ko = [i for i in nouns if i not in stopword]
ko

['도미니언',
 '영화포스터',
 '벌써',
 '시작',
 '오늘',
 '월',
 '부터',
 '실외',
 '마스크',
 '착용',
 '의무',
 '완화',
 '이후',
 '동안',
 '얼마나',
 '마스크',
 '착용',
 '완화',
 '기도',
 '계속',
 '마스크',
 '의견',
 '길거리',
 '착용',
 '사람',
 '모두',
 '안전',
 '위해',
 '건',
 '개인',
 '관리',
 '건강',
 '유의',
 '오늘',
 '포스팅',
 '시작',
 '역대',
 '대작',
 '쥬라기',
 '월드',
 '도미니언',
 '소개',
 '블랙위도우',
 '영화',
 '간략',
 '정보',
 '개봉',
 '등급',
 '세',
 '관람',
 '장르',
 '액션',
 '모험',
 '국가',
 '미국',
 '러닝',
 '타임',
 '줄거리',
 '네이버',
 '영화',
 '공룡',
 '터전',
 '이슬라',
 '누블',
 '섬',
 '파괴',
 '후',
 '마침내',
 '공룡',
 '섬',
 '세상',
 '밖',
 '출몰',
 '지상',
 '존재',
 '선',
 '안',
 '위협',
 '생명체',
 '공룡',
 '등장',
 '인류',
 '역사상',
 '보지',
 '못',
 '사상',
 '최악',
 '위기',
 '맞이',
 '인간',
 '지구',
 '최',
 '상위',
 '포식자',
 '자리',
 '인간',
 '공룡',
 '최후',
 '사투',
 '쥬라기',
 '월드',
 '도미니언',
 '영화포스터',
 '를',
 '강력',
 '이번',
 '포스팅',
 '마무리',
 '주문',
 '여기',
 '포토',
 '판다',
 '주문',
 '관련',
 '포스팅',
 '글',
 '돈',
 '잘버',
 '회사',
 '가치투자',
 '주식',
 '달라',
 '다음',
 '세대',
 '강태훈',
 '선생',
 '가정',
 '주식',
 '달라',
 '주식',
 '타령',
 '하니',
 '용돈',
 '주식',
 '미래',
 '부자',
 '이야기',
 '유태인',
 '세',
 '성인

In [67]:
#빈도수 체크
from collections import Counter
count = Counter(ko)
#count

#상위 3 출력
count.most_common(3)

[('장난감', 113), ('지갑', 102), ('보드', 75)]

https://dalulu.tistory.com/108
http://bigdata.dongguk.ac.kr/lectures/DB/_book/python%EC%97%90%EC%84%9C-mysql%EB%8D%B0%EC%9D%B4%ED%84%B0%EC%9D%98-%EC%A0%91%EA%B7%BC.html

In [71]:
#딕셔너리 생성 후 키값과 value값 분리
tag_count=[]
tags=[]

for n,c in count.most_common(3):
    dics={'tag': n, 'count':c}
    tag_count.append(dics)
    tags.append(dics['tag'])

https://hyun-am-coding.tistory.com/entry/%ED%81%AC%EB%A1%A4%EB%A7%81%ED%95%9C-%EB%8D%B0%EC%9D%B4%ED%84%B0-DB%EC%97%90-%EC%A0%80%EC%9E%A5%ED%95%98%EA%B8%B0
https://codechacha.com/ko/python-mysql-select-order/

In [77]:
!pip install mysqlclient
!pip install pymysql

In [105]:
#DB 셍성, 테이블 생성, 데이터 추가(처음 코드를 실행할 때 사용)

import MySQLdb

conn = MySQLdb.connect(
    user="crawl_usr",
    passwd="Test001",
    host="localhost",
    db="crawl_data"
    # charset="utf-8"
)
# 커서 생성하기
cursor = conn.cursor()

# 실행할 때마다 다른값이 나오지 않게 테이블을 제거해두기
cursor.execute("DROP TABLE IF EXISTS recommend")

# 테이블 생성하기
cursor.execute("CREATE TABLE recommend (`rank` int, situation text, gender text, age text, present text)")

# 데이터 저장하기
situation = "Children"    #Children, Parents, Teacher, Birthday, Wedding
gender = "woman"    #man, woman
age = "ten"    #ten, twenty, forty, sixty 
i=1
for tag in tag_count:
    cursor.execute(f"INSERT INTO recommend VALUES({i},\"{situation}\", \"{gender}\", \"{age}\", \"{tag['tag']}\")")
    i+=1
# 커밋하기
conn.commit()

# 연결종료하기
conn.close()

https://codechacha.com/ko/python-mysql-insert-update-delete/

In [104]:
#데이터 추가(DB 및 테이블 생성 후 데이터만 추가하기)

import pymysql

conn = MySQLdb.connect(
    user="crawl_usr",
    passwd="Test001",
    host="localhost",
    db="crawl_data"
)

# 커서 생성하기
cursor = conn.cursor()

sql = "INSERT INTO recommend (rank, situation, gender, age, present) VALUES (%d, %s, %s, %s, %s)"

# 데이터 저장하기
situation = "Children"
gender = "woman"
age = "ten"
i=1

for tag in tag_count:
    cursor.execute(f"INSERT INTO recommend VALUES({i},\"{situation}\", \"{gender}\", \"{age}\", \"{tag['tag']}\")")
    i+=1
# 커밋하기
conn.commit()

In [106]:
import pymysql

conn = MySQLdb.connect(
    user="crawl_usr",
    passwd="Test001",
    host="localhost",
    db="crawl_data"
)

# 테이블 불러옴
with conn:
    with conn.cursor() as cursor:
        #cursor.execute("SELECT * FROM recommend")
        cursor.execute("SELECT * FROM recommend WHERE situation LIKE %s", ['Children'])
        result=cursor.fetchall()
        print(result)


((1, 'Children', 'woman', 'ten', '장난감'), (2, 'Children', 'woman', 'ten', '지갑'), (3, 'Children', 'woman', 'ten', '보드'))
